# Importing Dependencies

In [9]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import pandas_datareader.data as web
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [5]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [6]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [4]:
from key import api_key, api_secret

In [7]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret)

[Errno 2] No such file or directory: 'tokens.json'
Token file does not exist or invalid formatting, creating "tokens.json"


[Schwabdev] Open to authenticate: https://api.schwabapi.com/v1/oauth/authorize?client_id=sLS2QjpxNJbE8Vp9AUABE1p8dc3EdIC5&redirect_uri=https://127.0.0.1


# Creating Datafram
- Basing this strategy in a 6 month change

In [22]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [10]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [23]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = ((price_now - price_6_months)/ price_6_months) * 100
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [24]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

In [25]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,MMM,124.75,133.445,6.969940,N/a
1,AOS,76.87,63.335,-17.607649,N/a
2,ABT,114.22,131.785,15.378217,N/a
3,ABBV,187.85,172.230,-8.315145,N/a
4,ACN,360.80,290.175,-19.574557,N/a
